# voronoi_pynq.ipynb

Author: Isaiah Hendrick

Project: Master's Project - PYNQ Voronoi Acceleration

Organization: Grand Valley State University

Description: This notebook demonstrates the hardware-accelerated computation of the Voronoi algorithm using a custom Vitis HLS IP overlay on the PYNQ-Z1 FPGA board. This code can only be run when interfaced with the PYNQ-Z1 board (accessed through web server), and must have appropriate ".bit" and ".hwh" files in same folder


### Import libraries

In [ ]:
from pynq import Overlay, allocate
import numpy as np
import matplotlib.pyplot as plt
import time

### Create the overlay and extract the IP

In [ ]:
ol = Overlay("voronoi.bit")
ip = ol.calculate_voronoi_0

### Allocate and write parameters to memory on PYNQ-Z1

In [ ]:
# Problem size
size = 4096

# Allocate space for result matrix
area_buf = allocate(shape=(size, size), dtype=np.float32)
area_buf.flush()

# Write values directly to physical memory of fpga
ip.write(0x10, size)
ip.write(0x18, area_buf.physical_address & 0xFFFFFFFF)
ip.write(0x1c, (area_buf.physical_address >> 32) & 0xFFFFFFFF)

### Call function on PYNQ-Z1

In [ ]:
# Record start time
start = time.time()

# Write to memory to start function call
ip.write(0x00, 0x01)
# Wait until function finishes
while (ip.read(0x00) & 0x2) == 0:
    pass
area_buf.invalidate()

# Record end time
end = time.time()

# Display timing results
elapsed = (end - start)
print("-------------------------------")
print("| PYNQ VORONOI IMPLEMENTATION |")
print("-------------------------------")
print(f"Array size: {size}\nTime taken: {elapsed} s")

### Verify results using matplotlib

In [ ]:
plt.imshow(area_buf, cmap='viridis')
plt.colorbar()
plt.show()

### Free allocated memory

In [ ]:
area_buf.freebuffer()

## Run C implementations for comparison

In [ ]:
!gcc voronoi_seq_c.c -o voronoi_seq_c -O3 -lm
!./voronoi_seq_c

In [ ]:
!gcc voronoi_omp.c -o voronoi_omp -fopenmp -O3 -lm
!OMP_NUM_THREADS=2 ./voronoi_omp